# Introduction to PySpark

### Index 

## Introduction

1. What is Spark?
2. Using Spark in Python
    - 2.1 Connecting to a cluster
3. Examining the SparkContext
4. Using DataFrames
    - 4.1 Creating a SparkSession
5. Creating a DataFrame from a local file 
6. Creating a DataFrame from a RDD 

## Manipulating data

7. Creating columns
8. Renaming columns
9. Selecting columns 
10. Creating columns with the .select() method
11. Filtering
12. Joins 
13. Aggregating

## Using SQL 

14. SQL reminder
15. Pandafy a Spark DataFrame
16. Put some Spark in your data
17. Viewing tables

## 1. What is Spark?

Spark is a platform for cluster computing. Spark lets you spread data and computations over clusters with multiple nodes (think of each node as a separate computer). Splitting up your data makes it easier to work with very large datasets because each node only works with a small amount of data.

As each node works on its own subset of the total data, it also carries out a part of the total calculations required, so that both data processing and computation are performed in parallel over the nodes in the cluster. It is a fact that parallel computation can make certain types of programming tasks much faster.

---

## 2. Using Spark in Python

### 2.1 Connecting to a cluster

The first step in using Spark is connecting to a cluster. In practice, the cluster will be hosted on a remote machine that's connected to all other nodes. There will be one computer, called the **master** (or driver) that manages splitting up the data and the computations. The master is connected to the rest of the computers in the cluster, which are called **workers**. The master sends the workers data and calculations to run, and they send their results back to the master.

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.sql.types import StructType, StructField 
from pyspark.sql.types import IntegerType, StringType, FloatType

from pyspark.sql.functions import *

## 3. Examining The SparkContext

Creating the connection is as simple as creating an instance of the `SparkContext` class. The class constructor takes a few optional arguments that allow you to specify the attributes of the cluster you're connecting to. Take a look at the [documentation](http://spark.apache.org/docs/2.1.0/api/python/pyspark.html) for all the details!

In [2]:
sc = SparkContext(master = "local", appName = "Introduction to pySpark") 

# Verify SparkContext
print(sc)
# Print Spark version
print(sc.version)

You'll probably notice that code takes longer to run than you might expect. This is because Spark is some serious software. It takes more time to start up than you might be used to. You may also find that running simpler computations might take longer than expected. That's because all the optimizations that Spark has under its hood are designed for complicated operations with big data sets. That means that for simple or small problems Spark may actually perform worse than some other solutions!

## 4. Using DataFrames

Spark's core data structure is the **Resilient Distributed Dataset** (RDD). This is a low level object that lets Spark work its magic by splitting data across multiple nodes in the cluster. However, RDDs are hard to work with directly, so we'll be using the Spark `DataFrame` abstraction built on top of RDDs.

The Spark `DataFrame` was designed to behave a lot like a SQL table (a table with variables in the columns and observations in the rows). Not only are they easier to understand, `DataFrames` are also more optimized for complicated operations than RDDs.

When you start modifying and combining columns and rows of data, there are many ways to arrive at the same result, but some often take much longer than others. When using `RDD`s, it's up to the data scientist to figure out the right way to optimize the query, but the `DataFrame` implementation has much of this optimization built in!

### 4.1 Creating a SparkSession

To start working with Spark `DataFrames`, you first have to create a `SparkSession` object from your `SparkContext`. You can think of the `SparkContext` as your connection to the cluster and the `SparkSession` as your interface with that connection.

In [4]:
spark = SparkSession(sc)
sqlContext = SQLContext(spark)

In [5]:
# Files we will be working with
path = "/home/danae/Documents/pySparkTraining/files/"

!ls /home/danae/Documents/pySparkTraining/files

AA_DFW_2014_Departures_Short.csv  DallasCouncilVotes.csv  juegos.csv
AA_DFW_2015_Departures_Short.csv  deporte.csv		  modelo_relacional.jpg
AA_DFW_2016_Departures_Short.csv  deportista2.csv	  paises.csv
AA_DFW_2017_Departures_Short.csv  deportista.csv	  people.csv
cheatsheet_spark.pdf		  deportistaError.csv	  resultados.csv
DallasCouncilVoters.csv		  evento.csv


We've already created a `SparkSession` called `spark`, but what if you're not sure there already is one? Creating multiple `SparkSession`s and `SparkContext`s can cause issues, so it's best practice to use the `SparkSession.builder.getOrCreate()` method. This returns an existing `SparkSession` if there's already one in the environment, or creates a new one if necessary!

In [6]:
# Create a spark session 
spark = SparkSession.builder.getOrCreate()

# Print spark
print(spark)

## 5. Creating a DataFrame from a local file 

Your SparkSession has a `.read` attribute which has several methods for reading different data sources into Spark `DataFrames`. Using these you can create a DataFrame from a `.csv` file just like with regular pandas DataFrames!

In [7]:
sport = spark.read.csv(path + 'deporte.csv', header = True)

# Show the data
sport.show(5)

+----------+-------------+
|deporte_id|      deporte|
+----------+-------------+
|         1|   Basketball|
|         2|         Judo|
|         3|     Football|
|         4|   Tug-Of-War|
|         5|Speed Skating|
+----------+-------------+
only showing top 5 rows



In [8]:
sport.printSchema()

root
 |-- deporte_id: string (nullable = true)
 |-- deporte: string (nullable = true)



You can achive the same results and also specify a schema with the `sqlContext.read.schema` function in order to specify the type of column. 

In [9]:
sportSchema = StructType([
    StructField('sport_id', IntegerType(), False),
    StructField('sport', StringType(), False)
])

sportDF = sqlContext.read.schema(sportSchema) \
            .option('header', 'true').csv(path + 'deporte.csv')

sportDF.show(5)

+--------+-------------+
|sport_id|        sport|
+--------+-------------+
|       1|   Basketball|
|       2|         Judo|
|       3|     Football|
|       4|   Tug-Of-War|
|       5|Speed Skating|
+--------+-------------+
only showing top 5 rows



In [10]:
eventSchema = StructType([
    StructField('event_id', IntegerType(), False),
    StructField('name', StringType(), False),
    StructField('sport_id', IntegerType(), False),    
])

OlimpicSportsDF = sqlContext.read.schema(eventSchema) \
            .option('header', 'true').csv(path + 'evento.csv')

OlimpicSportsDF.show(5)

+--------+--------------------+--------+
|event_id|                name|sport_id|
+--------+--------------------+--------+
|       1|Basketball Men's ...|       1|
|       2|Judo Men's Extra-...|       2|
|       3|Football Men's Fo...|       3|
|       4|Tug-Of-War Men's ...|       4|
|       5|Speed Skating Wom...|       5|
+--------+--------------------+--------+
only showing top 5 rows



In [11]:
gameSchema = StructType([
    StructField('game_id', IntegerType(), False),
    StructField('name_game', StringType(), False),
    StructField('year', StringType(), False),
    StructField('season', StringType(), False),
    StructField('city', StringType(), False)
])

gameDF = sqlContext.read.schema(gameSchema) \
            .option('header', 'true').csv(path + 'juegos.csv')

gameDF.show(5)

+-------+-----------+----+------+---------+
|game_id|  name_game|year|season|     city|
+-------+-----------+----+------+---------+
|      1|1896 Verano|1896|Verano|   Athina|
|      2|1900 Verano|1900|Verano|    Paris|
|      3|1904 Verano|1904|Verano|St. Louis|
|      4|1906 Verano|1906|Verano|   Athina|
|      5|1908 Verano|1908|Verano|   London|
+-------+-----------+----+------+---------+
only showing top 5 rows



In [12]:
teamSchema = StructType([
    StructField('id', IntegerType(), False),
    StructField('team', StringType(), False),
    StructField('country', StringType(), False),
])

countryDF = sqlContext.read.schema(teamSchema) \
            .option('header', 'true').csv(path + 'paises.csv')

countryDF.show(5)

+---+--------------------+-------+
| id|                team|country|
+---+--------------------+-------+
|  1|         30. Februar|    AUT|
|  2|A North American ...|    MEX|
|  3|           Acipactli|    MEX|
|  4|             Acturus|    ARG|
|  5|         Afghanistan|    AFG|
+---+--------------------+-------+
only showing top 5 rows



In [13]:
resultSchema = StructType([
    StructField('result_id', IntegerType(), False),
    StructField('medal', StringType(), False),
    StructField('athlete_id', IntegerType(), False),
    StructField('game_id', IntegerType(), False),
    StructField('event_id', IntegerType(), False),    
])

resultDF = sqlContext.read.schema(resultSchema) \
            .option('header', 'true').csv(path + 'resultados.csv')

resultDF.show(5)

+---------+-----+----------+-------+--------+
|result_id|medal|athlete_id|game_id|event_id|
+---------+-----+----------+-------+--------+
|        1|   NA|         1|     39|       1|
|        2|   NA|         2|     49|       2|
|        3|   NA|         3|      7|       3|
|        4| Gold|         4|      2|       4|
|        5|   NA|         5|     36|       5|
+---------+-----+----------+-------+--------+
only showing top 5 rows



## 6. Creating a DataFrame from a RDD 

In [14]:
athleteRDD = sc.textFile(path +'deportista.csv') \
    .map(lambda line: line.split(","))

athleteRDD2 = sc.textFile(path + 'deportista2.csv') \
    .map(lambda line: line.split(","))

athleteRDD = athleteRDD.union(athleteRDD2) # operations using RDDs
athleteRDD.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [15]:
athleteRDD.count()

135572

In [16]:
def deleteHeadline(index, iterator):
    return iter(list(iterator)[1:])

athleteRDD = athleteRDD.mapPartitionsWithIndex(deleteHeadline)
athleteRDD.take(5)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

In [17]:
athleteRDD = athleteRDD.map(lambda l: (int(l[0]),l[1],int(l[2]),int(l[3]),int(l[4]),float(l[5]),int(l[6])))
athleteRDD.take(5)

[(1, 'A Dijiang', 1, 24, 180, 80.0, 199),
 (2, 'A Lamusi', 1, 23, 170, 60.0, 199),
 (3, 'Gunnar Nielsen Aaby', 1, 24, 0, 0.0, 273),
 (4, 'Edgar Lindenau Aabye', 1, 34, 0, 0.0, 278),
 (5, 'Christine Jacoba Aaftink', 2, 21, 185, 82.0, 705)]

In [18]:
# create a DataFrame from the RDD 
schema = StructType([
    StructField('athlete_id', IntegerType(), False),
    StructField('name', StringType(), False),
    StructField('gender', IntegerType(), False),
    StructField('age', IntegerType(), False),
    StructField('height', IntegerType(), False),
    StructField('weigth', FloatType(), False),
    StructField('team_id', IntegerType(), False)
])

athleteDF = sqlContext.createDataFrame(athleteRDD, schema)
athleteDF.show(5)

+----------+--------------------+------+---+------+------+-------+
|athlete_id|                name|gender|age|height|weigth|team_id|
+----------+--------------------+------+---+------+------+-------+
|         1|           A Dijiang|     1| 24|   180|  80.0|    199|
|         2|            A Lamusi|     1| 23|   170|  60.0|    199|
|         3| Gunnar Nielsen Aaby|     1| 24|     0|   0.0|    273|
|         4|Edgar Lindenau Aabye|     1| 34|     0|   0.0|    278|
|         5|Christine Jacoba ...|     2| 21|   185|  82.0|    705|
+----------+--------------------+------+---+------+------+-------+
only showing top 5 rows



# Manipulating data

### Index 
7. Creating columns
8. Renaming columns
9. Selecting columns 
10. Creating columns with the .select() method
11. Filtering
12. Joins 
13. Aggregating

---

## 7. Creating columns

Let's explore the methods defined by Spark's `DataFrame` class to perform common data operations.

For performing column-wise operations in Spark you can use the **`.withColumn()`** method, which takes two arguments. First, a string with the name of your new column, and second the new column itself.

The new column must be an object of class `Column`. Creating one of these is as easy as extracting a column from your `DataFrame` using the syntax `df.colName`.

In [19]:
athleteDF = athleteDF.withColumn("weigth_pounds", athleteDF.weigth*2.2)
athleteDF.show(5)

+----------+--------------------+------+---+------+------+-------+-------------+
|athlete_id|                name|gender|age|height|weigth|team_id|weigth_pounds|
+----------+--------------------+------+---+------+------+-------+-------------+
|         1|           A Dijiang|     1| 24|   180|  80.0|    199|        176.0|
|         2|            A Lamusi|     1| 23|   170|  60.0|    199|        132.0|
|         3| Gunnar Nielsen Aaby|     1| 24|     0|   0.0|    273|          0.0|
|         4|Edgar Lindenau Aabye|     1| 34|     0|   0.0|    278|          0.0|
|         5|Christine Jacoba ...|     2| 21|   185|  82.0|    705|        180.4|
+----------+--------------------+------+---+------+------+-------+-------------+
only showing top 5 rows



The above code creates a `DataFrame` with the same columns as `athleteDF` plus a new column, `weigth_pounds`, where every entry is equal to the corresponding entry from `weigth`, multiplied by 2.2

Updating a Spark `DataFrame` is somewhat different than working in pandas because the Spark `DataFrame` is immutable. This means that it can't be changed, and so columns can't be updated in place.

Thus, all these methods return a new `DataFrame`. To overwrite the original `DataFrame` you must reassign the returned `DataFrame` using the method like so:

`df = df.withColumn("newCol", df.oldCol + 1)`

--- 

## 8. Renaming columns

To rename columns you can use the **`.withColumnRenamed()`** method, which takes two arguments, the name of the column we want to replace and the new column name we want to asign. 

In [20]:
# renombrado de columnas
athleteDF = athleteDF.withColumnRenamed("gender", 'sex')
athleteDF.show(5)

+----------+--------------------+---+---+------+------+-------+-------------+
|athlete_id|                name|sex|age|height|weigth|team_id|weigth_pounds|
+----------+--------------------+---+---+------+------+-------+-------------+
|         1|           A Dijiang|  1| 24|   180|  80.0|    199|        176.0|
|         2|            A Lamusi|  1| 23|   170|  60.0|    199|        132.0|
|         3| Gunnar Nielsen Aaby|  1| 24|     0|   0.0|    273|          0.0|
|         4|Edgar Lindenau Aabye|  1| 34|     0|   0.0|    278|          0.0|
|         5|Christine Jacoba ...|  2| 21|   185|  82.0|    705|        180.4|
+----------+--------------------+---+---+------+------+-------+-------------+
only showing top 5 rows



## 9. Selecting columns 

The `.select` method takes multiple arguments - one for each column you want to select. These arguments can either be the column name as a **string** (one for each column) or a column object (using the `df.colName` syntax). 

When you pass a column object, you can perform operations like addition or subtraction on the column to change the data contained in it, much like inside `.withColumn()`.

In [21]:
athleteDF.printSchema()

root
 |-- athlete_id: integer (nullable = false)
 |-- name: string (nullable = false)
 |-- sex: integer (nullable = false)
 |-- age: integer (nullable = false)
 |-- height: integer (nullable = false)
 |-- weigth: float (nullable = false)
 |-- team_id: integer (nullable = false)
 |-- weigth_pounds: double (nullable = false)



In [22]:
athleteDF2 = athleteDF.select(athleteDF.athlete_id
                              , athleteDF.name
                              , athleteDF.age
                              )
athleteDF2.show(5)

+----------+--------------------+---+
|athlete_id|                name|age|
+----------+--------------------+---+
|         1|           A Dijiang| 24|
|         2|            A Lamusi| 23|
|         3| Gunnar Nielsen Aaby| 24|
|         4|Edgar Lindenau Aabye| 34|
|         5|Christine Jacoba ...| 21|
+----------+--------------------+---+
only showing top 5 rows



In [23]:
athleteDF3 = athleteDF.select("athlete_id"
                              , "name"
                              , "age"
                              )
athleteDF3.show(5)

+----------+--------------------+---+
|athlete_id|                name|age|
+----------+--------------------+---+
|         1|           A Dijiang| 24|
|         2|            A Lamusi| 23|
|         3| Gunnar Nielsen Aaby| 24|
|         4|Edgar Lindenau Aabye| 34|
|         5|Christine Jacoba ...| 21|
+----------+--------------------+---+
only showing top 5 rows



The difference between `.select()` and `.withColumn()` methods is that `.select()` returns only the columns you specify, while `.withColumn()` returns all the columns of the DataFrame in addition to the one you defined. 

--- 

It's often a good idea to drop columns you don't need at the beginning of an operation so that you're not dragging around extra data as you're wrangling. In this case, you would use `.select()` and not `.withColumn()`.

--- 

## 10. Creating columns with the .select() method

Similar to SQL, you can also use the `.select()` method to perform column-wise operations. When you're selecting a column using the `df.colName` notation, you can perform any column operation and the `.select()` method will return the transformed column. 

You can also use the `.alias()` method to rename a column you're selecting. 

In [24]:
athleteBMI = athleteDF.select(athleteDF.height
                            , athleteDF.weigth
                            , ((athleteDF.weigth/athleteDF.height)**2).alias("BMI"))
athleteBMI.show(5)

+------+------+-------------------+
|height|weigth|                BMI|
+------+------+-------------------+
|   180|  80.0|0.19753086419753085|
|   170|  60.0|0.12456747404844293|
|     0|   0.0|               null|
|     0|   0.0|               null|
|   185|  82.0|0.19646457268078893|
+------+------+-------------------+
only showing top 5 rows



The equivalent Spark `DataFrame` method `.selectExpr()` takes SQL expressions as a string with the SQL `as` keyword being equivalent to the `.alias()` method. To select multiple columns, you can pass multiple strings.

In [25]:
BMI = athleteDF.selectExpr("power(weigth/height,2) as BMI")
BMI.show(5)

+-------------------+
|                BMI|
+-------------------+
|0.19753086419753085|
|0.12456747404844293|
|               null|
|               null|
|0.19646457268078893|
+-------------------+
only showing top 5 rows



In [26]:
sub_athleteDF = athleteDF.select("athlete_id"
                             , "name"
                             , col("age").alias("age_in_game")
                             ,"team_id")
sub_athleteDF.show(5)

+----------+--------------------+-----------+-------+
|athlete_id|                name|age_in_game|team_id|
+----------+--------------------+-----------+-------+
|         1|           A Dijiang|         24|    199|
|         2|            A Lamusi|         23|    199|
|         3| Gunnar Nielsen Aaby|         24|    273|
|         4|Edgar Lindenau Aabye|         34|    278|
|         5|Christine Jacoba ...|         21|    705|
+----------+--------------------+-----------+-------+
only showing top 5 rows



## 11. Filtering

The `.filter()` method takes a Spark Column combined with a logical expression generating boolean (True/False) values.

In [27]:
sub_athleteDF2 = sub_athleteDF.filter(sub_athleteDF.age_in_game != 0)
sub_athleteDF2.show(5)

+----------+--------------------+-----------+-------+
|athlete_id|                name|age_in_game|team_id|
+----------+--------------------+-----------+-------+
|         1|           A Dijiang|         24|    199|
|         2|            A Lamusi|         23|    199|
|         3| Gunnar Nielsen Aaby|         24|    273|
|         4|Edgar Lindenau Aabye|         34|    278|
|         5|Christine Jacoba ...|         21|    705|
+----------+--------------------+-----------+-------+
only showing top 5 rows



Spark's `.filter()` can accept any expression that could go in the `WHERE` clause of a SQL query, **as long as it is passed as a string**. Notice that in this case, we do not reference the name of the table in the string.

In [28]:
sub_athleteDF3 = sub_athleteDF.filter("age_in_game <> 0")
sub_athleteDF3.show(5)

+----------+--------------------+-----------+-------+
|athlete_id|                name|age_in_game|team_id|
+----------+--------------------+-----------+-------+
|         1|           A Dijiang|         24|    199|
|         2|            A Lamusi|         23|    199|
|         3| Gunnar Nielsen Aaby|         24|    273|
|         4|Edgar Lindenau Aabye|         34|    278|
|         5|Christine Jacoba ...|         21|    705|
+----------+--------------------+-----------+-------+
only showing top 5 rows



In [29]:
sub_athleteDF2.sort("age_in_game").show(5)

+----------+--------------------+-----------+-------+
|athlete_id|                name|age_in_game|team_id|
+----------+--------------------+-----------+-------+
|     71691|  Dimitrios Loundras|         10|    333|
|     52070|        Etsuko Inada|         11|    514|
|     40129|    Luigina Giavotti|         11|    507|
|     37333|Carlos Bienvenido...|         11|    982|
|     47618|Sonja Henie Toppi...|         11|    742|
+----------+--------------------+-----------+-------+
only showing top 5 rows



## 12. Joins 

A join will combine two different tables along a column that they share. This column is called the key.

In PySpark, joins are performed using the DataFrame method `.join()`. This method takes three arguments. The first is the second DataFrame that you want to join with the first one. The second argument, `on`, is the name of the key column(s) as a string, (but for using this syntax, the names of the key column(s) must be the same in each table). The third argument, `how`, specifies the kind of join to perform.

If the tables you want to join have different key names, you can use the following syntax:

In [30]:
medal_per_year = athleteDF.join(resultDF, athleteDF.athlete_id == resultDF.athlete_id, "left") \
                          .join(gameDF, gameDF.game_id == resultDF.game_id,  "left") \
                          .join(countryDF, athleteDF.team_id == countryDF.id, "left") \
                          .join(sportDF, sportDF.sport_id == resultDF.event_id, "left") \
                          .select("country", "name", "sport", "year", "medal")
    
medal_per_year.show(3)

+-------+-------------------+----------+----+-----+
|country|               name|     sport|year|medal|
+-------+-------------------+----------+----+-----+
|    CHN|          A Dijiang|Basketball|1992|   NA|
|    CHN|           A Lamusi|      Judo|2012|   NA|
|    DEN|Gunnar Nielsen Aaby|  Football|1920|   NA|
+-------+-------------------+----------+----+-----+
only showing top 3 rows



## 13. Aggregating

All of the common aggregation methods, like `.min()`, `.max()`, and `.count()` are `GroupedData` methods. These are created by calling the `.groupBy()` DataFrame method. 

For example, to find the minimum value of a column, `col`, in a DataFrame, `df`, you could do

`df.groupBy().min("col").show()`

This creates a `GroupedData` object (so you can use the `.min()` method), then finds the minimum value in `col`, and returns it as a DataFrame.

In [31]:
athleteDF.groupBy().avg("age").show()

+------------------+
|          avg(age)|
+------------------+
|23.299586929261636|
+------------------+



Another way to achive this, is by using the `.agg()` method with a dictionary indicating the column and the function (any of the aggregate functions from the `pyspark.sql.functions` submodule) you want to apply. 

In [32]:
mean_age = athleteDF.agg({'age': 'mean'}).collect()[0][0]
mean_age

23.299586929261636

When you pass the name of one or more columns in your DataFrame to the `.groupBy()` method, the aggregation methods behave like when you use a `GROUP BY` statement in a SQL query!

In [33]:
# another more complex example
# remove athets without medals 
medal_per_year2 = medal_per_year.filter(medal_per_year.medal != "NA") \
                    .sort("year") \
                    .groupBy("country", "year", "sport") \
                    .count()

medal_per_year2.show(5)

+-------+----+----------------+-----+
|country|year|           sport|count|
+-------+----+----------------+-----+
|    USA|1896|         Curling|    1|
|    FRA|1896|            Polo|    2|
|    USA|1896|       Triathlon|    3|
|    GRE|1896|Beach Volleyball|    2|
|    AUS|1896|         Curling|    1|
+-------+----+----------------+-----+
only showing top 5 rows



In [34]:
medal_per_year2.filter(medal_per_year2.country == "USA").groupBy().max("count").show()

+----------+
|max(count)|
+----------+
|       295|
+----------+



In [35]:
medal_per_year2.filter(medal_per_year2.country == "CAN").groupBy().max("count").show()

+----------+
|max(count)|
+----------+
|        77|
+----------+



In [36]:
medal_per_year2.filter(medal_per_year2.country == "MEX").groupBy().max("count").show()

+----------+
|max(count)|
+----------+
|        16|
+----------+



In [37]:
medal_per_year2.groupBy("country", "year")\
        .agg(sum("count").alias("Total"), avg("count").alias("Average"))\
        .show(5)

+-------+----+-----+------------------+
|country|year|Total|           Average|
+-------+----+-----+------------------+
|    USA|1896|   20|               4.0|
|    FRA|1896|   11|3.6666666666666665|
|    GRE|1896|   48|               9.6|
|    AUS|1896|    3|               1.5|
|    GER|1896|   32| 5.333333333333333|
+-------+----+-----+------------------+
only showing top 5 rows



# Using SQL 

### Index 
14. SQL reminder
15. Pandafy a Spark DataFrame
16. Put some Spark in your data
17. Viewing tables

One of the advantages of the `DataFrame` interface is that you can run SQL queries on the tables in your Spark cluster. 

Running a query on a table is as easy as using the `.sql()` method on your `SparkSession`. This method takes a string containing the query and returns a `DataFrame` with the results!

In [38]:
# Assign an alias so that it can be used with SQL 
countryDF.registerTempTable("country") 

# write your query as a string 
query = "FROM country SELECT * LIMIT 5"

# Get the first 5 rows of countryDF
country5 = sqlContext.sql(query)

# Show the results
country5.show()

+---+--------------------+-------+
| id|                team|country|
+---+--------------------+-------+
|  1|         30. Februar|    AUT|
|  2|A North American ...|    MEX|
|  3|           Acipactli|    MEX|
|  4|             Acturus|    ARG|
|  5|         Afghanistan|    AFG|
+---+--------------------+-------+



## 14. SQL reminder

A SQL query returns a table derived from one or more tables contained in a database. The two commands that every query has to contain are `SELECT` and `FROM`.The minimal SQL query is:

`SELECT * FROM my_table;`

The `*` selects all columns, so this returns the entire table named `my_table`. Similar to `.withColumn()`, you can do column-wise computations within a `SELECT` statement. For example,

`SELECT weigth, team_id, weigth*2.2 FROM athletesDF `

returns a table with the `weigth`, `team_id` and weigth in pounds each athlete. Another commonly used command is `WHERE`. This command filters the rows of the table based on some logical condition you specify. The resulting table contains the rows where your condition is true. For example, if you had a table of students and grades you could do:

`SELECT * FROM students WHERE grade = 'A';`

to select all the columns and the rows containing information about students who got As. Another common database task is aggregation. That is, reducing your data by breaking it into chunks and summarizing each chunk.

This is done in SQL using the `GROUP BY` command. This command breaks your data into groups and applies a function from your `SELECT` statement to each group. For example, if you wanted to count the number of athletes from each country, you could use the query

`SELECT COUNT(*) FROM athletesDF GROUP BY country;`

`GROUP BY` origin tells SQL that you want the output to have a row for each unique value of the country column. The `SELECT` statement selects the values you want to populate each of the columns. Here, we want to `COUNT()` every row in each of the groups.

It's possible to `GROUP BY` more than one column. When you do this, the resulting table has a row for every combination of the unique values in each column. 

In [39]:
resultDF.registerTempTable("result")
sqlContext.sql("SELECT * FROM result").show(5)

+---------+-----+----------+-------+--------+
|result_id|medal|athlete_id|game_id|event_id|
+---------+-----+----------+-------+--------+
|        1|   NA|         1|     39|       1|
|        2|   NA|         2|     49|       2|
|        3|   NA|         3|      7|       3|
|        4| Gold|         4|      2|       4|
|        5|   NA|         5|     36|       5|
+---------+-----+----------+-------+--------+
only showing top 5 rows



## 15. Pandafy a Spark DataFrame

Suppose you've run a query on your huge dataset and aggregated it down to something a little more manageable.

Sometimes it makes sense to then take that table and work with it locally using a tool like `pandas`. Spark `DataFrames` make that easy with the `.toPandas()` method. Calling this method on a Spark `DataFrame` returns the corresponding pandas `DataFrame`. It's as simple as that!

In [40]:
sportDF.registerTempTable("sport")
sqlContext.sql("SELECT * FROM sport").show(5)

+--------+-------------+
|sport_id|        sport|
+--------+-------------+
|       1|   Basketball|
|       2|         Judo|
|       3|     Football|
|       4|   Tug-Of-War|
|       5|Speed Skating|
+--------+-------------+
only showing top 5 rows



In [41]:
query = "SELECT * FROM sport"
# Run the query
pd_sports = spark.sql(query)
# Convert the results to a pandas DataFrame
pd_sports = pd_sports.toPandas()
# Print the head of pd_counts
pd_sports.head()

,sport_id,sport
0,1,Basketball
1,2,Judo
2,3,Football
3,4,Tug-Of-War
4,5,Speed Skating


From now on you can use the regular functions you know from `pandas`! 

--- 

## 16. Put some Spark in your data

We already saw how to move data from Spark to `pandas`. However, maybe you want to go the other direction, and put a `pandas` DataFrame into a Spark cluster! The `SparkSession` class has a method for this as well.

The `.createDataFrame()` method takes a `pandas` DataFrame and returns a Spark DataFrame.

The output of this method is stored locally, not in the `SparkSession` catalog. This means that you can use all the Spark DataFrame methods on it, but you can't access the data in other contexts.

For example, a SQL query (using the `.sql()` method) that references your DataFrame will throw an error. To access the data in this way, you have to save it as a temporary table. (as we did we the tables above).

You can do this using the `.createTempView()` Spark DataFrame method, which takes as its only argument the name of the temporary table you'd like to register. This method registers the DataFrame as a table in the catalog, but as this table is temporary, it can only be accessed from the specific `SparkSession` used to create the Spark DataFrame.

There is also the method `.createOrReplaceTempView()`. This safely creates a new temporary table if nothing was there before, or updates an existing table if one was already defined. You can use this method to avoid running into problems with duplicate tables.

In [42]:
import numpy as np 
import pandas as pd 
# Create pd_temp
pd_temp = pd.DataFrame(np.random.random(10))

# Create spark_temp from pd_temp
spark_temp = spark.createDataFrame(pd_temp)

## 17. Viewing tables

Your `SparkSession` has an attribute called `catalog` which lists all the data inside the cluster. This attribute has a few methods for extracting different pieces of information.

One of the most useful is the `.listTables()` method, which returns the names of all the tables in your cluster as a list.

Remark that the output of the `sqlContext.read.schema` method is stored locally, not in the `SparkSession` catalog. This means that you can use all the Spark DataFrame methods on it, but you can't access the data in other contexts.


In [43]:
# Add spark_temp to the catalog
spark_temp.createOrReplaceTempView("temp")

# Examine the tables in the catalog again
print(spark.catalog.listTables())

[Table(name='country', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='result', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='sport', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [44]:
sc.stop() # close the spark session